In [73]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace
from pyspark.sql.types import IntegerType
import pandas
import psycopg2

In [74]:
scSpark = SparkSession \
        .builder \
        .appName("simpleetl") \
        .getOrCreate()


In [75]:
df = scSpark \
    .read \
    .format("csv") \
    .option("header","true") \
    .load("autos.csv")

In [76]:
df.toPandas().head(5)

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_T�V_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [77]:
df.count()

50000

In [78]:
df.printSchema()

root
 |-- dateCrawled: string (nullable = true)
 |-- name: string (nullable = true)
 |-- seller: string (nullable = true)
 |-- offerType: string (nullable = true)
 |-- price: string (nullable = true)
 |-- abtest: string (nullable = true)
 |-- vehicleType: string (nullable = true)
 |-- yearOfRegistration: string (nullable = true)
 |-- gearbox: string (nullable = true)
 |-- powerPS: string (nullable = true)
 |-- model: string (nullable = true)
 |-- odometer: string (nullable = true)
 |-- monthOfRegistration: string (nullable = true)
 |-- fuelType: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- notRepairedDamage: string (nullable = true)
 |-- dateCreated: string (nullable = true)
 |-- nrOfPictures: string (nullable = true)
 |-- postalCode: string (nullable = true)
 |-- lastSeen: string (nullable = true)



In [79]:
df_cl = df.dropna()

In [80]:
df.toPandas().head(5)

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_T�V_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [81]:
df_dr = df_cl.drop("seller","offerType","dateCrawled","nrOfPictures","lastSeen")

In [82]:
df_rprice = df_dr.withColumn('price', regexp_replace('price', '[$,]', ''))

In [83]:
df_rodo = df_rprice.withColumn('odometer', regexp_replace('odometer', '[,km]', ''))

In [84]:
df_dt = df_rodo.withColumn("price", df_rodo['price'].cast(IntegerType()))

In [85]:
df_dt2 = df_dt.withColumn("odometer", df_dt['odometer'].cast(IntegerType()))

In [86]:
df_final = df_dt2

In [87]:
df_final.count()

35047

In [88]:
conn = psycopg2.connect(
    host = "localhost",
    database = "cars",
    user = "postgres",
    password = "admin"
)

In [89]:
cur = conn.cursor()

In [90]:
cur.execute("create table if not exists cars_table \
    (name varchar(255) not null, \
    price integer not null, \
    abtest varchar(255) not null, \
    vehicleType varchar(255), \
    yearOfRegistration varchar(4) not null, \
    gearbox varchar(255), \
    powerPS integer not null, \
    model varchar(255), \
    kilometer integer, \
    monthOfRegistration varchar(255) not null, \
    fuelType varchar(255), \
    brand varchar(255) not null, \
    notRepairedDamage varchar(255), \
    dateCreated date not null, \
    postalCode varchar(255) not null);")

conn.commit()

In [114]:
cars_seq = [tuple(x) for x in df_final.collect()]

cols = ','.join(list(df_final.columns))
# records_list_template = ','.join(['%$'] * len(cars_seq))

insert_query = "insert into cars_table(%$) values (%%$,%%$,%%$,%%$,%%$,%%$,%%$,%%$,%%$,%%$,%%$,%%$,%%$,%%$,%%$)" % (cols)

ValueError: unsupported format character '$' (0x24) at index 24

In [110]:
print(insert_query)

insert into cars_table (%$) values (%%$,%%$,%%$,%%$,%%$,%%$,%%$,%%$,%%$,%%$,%%$,%%$,%%$,%%$,%%$)


In [111]:
cur.executemany(insert_query,cars_seq)
conn.commit()

ValueError: unsupported format character '$' (0x24) at index 25

In [ ]:
conn.close()